# 💳 Detecção de Fraudes em Transações Financeiras com Machine Learning
### Disciplina: Engenharia de Machine Learning

**Equipe:**  
- Anderson de Matos Guimarães  
- Gustavo Stefano Thomazinho  
- Leonardo Rodrigues Vianna de Medeiros Lopes  
- Renan Ost  

**Professor:** Marcelo Carboni Gomes  
**Metodologia:** CRISP-DM  
**Dataset:** [Credit Card Fraud Detection (Kaggle)](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)


## 1. Entendimento do Negócio

Este projeto tem como objetivo desenvolver um modelo preditivo capaz de detectar transações fraudulentas com cartão de crédito, utilizando algoritmos de aprendizado de máquina supervisionado.

Fraudes financeiras representam um desafio para instituições bancárias, pois causam prejuízos significativos e minam a confiança dos clientes. O foco deste projeto é **maximizar a detecção de transações fraudulentas** (alta taxa de recall) sem comprometer excessivamente a taxa de falsos positivos.

A base de dados utilizada é real, anonimizadas por PCA, e extremamente desbalanceada — apenas **0,172% das transações são fraudes**. Isso impõe desafios na modelagem e na avaliação de desempenho dos modelos.

A metodologia adotada será o **CRISP-DM**, seguindo as etapas: entendimento do negócio, entendimento dos dados, preparação, modelagem, avaliação e apresentação.


### 1.1 Documentação Técnica do Dataset

Esta seção apresenta a documentação técnica detalhada do dataset utilizado no projeto, conforme exigido pelo professor:

#### 1) Rótulos (nomes das variáveis)

- `Time`: tempo (em segundos) desde a primeira transação registrada.
- `Amount`: valor da transação em euros (€).
- `V1` a `V28`: componentes principais resultantes de uma transformação PCA aplicada para anonimização dos dados originais. Seus significados exatos não são públicos, mas mantêm relevância estatística.
- `Class`: variável-alvo. Valores:
  - `0` → transação legítima
  - `1` → transação fraudulenta

#### 2) Tipos de dados

| Coluna        | Tipo de dado |
|---------------|--------------|
| `Time`        | `float64`    |
| `Amount`      | `float64`    |
| `V1` a `V28`  | `float64`    |
| `Class`       | `int64`      |

> Obs.: O dataset não contém valores ausentes (NaN) nem temporais no formato `datetime` (NaT).

#### 3) Quantitativos

- **Total de registros (linhas):** 284.807
- **Total de variáveis (colunas):** 31
- **Total de transações fraudulentas (`Class = 1`):** 492  
- **Proporção de fraudes:** aproximadamente 0,172%
- **Transações legítimas (`Class = 0`):** 284.315

#### 4) Número de datasets

- Apenas **um dataset** está sendo utilizado neste projeto:
  - `creditcard.csv`

#### 5) Relacionamentos

- O dataset é **auto contido** (flat table).
- Não há relacionamento com outras tabelas ou bases externas.
- Cada linha representa uma transação financeira independente.

#### 6) Formato dos dados

- **Formato:** `CSV` (Comma-Separated Values)
- **Codificação:** UTF-8
- **Fonte oficial:** Kaggle  
  [Credit Card Fraud Detection – Kaggle: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)

> Este dataset foi coletado em uma parceria entre o grupo Worldline e o Machine Learning Group da Université Libre de Bruxelles (ULB), e contempla transações realizadas por portadores de cartões europeus em setembro de 2013.


## 🔧 Preparação Inicial

Nesta célula, importamos as bibliotecas necessárias para a análise e carregamos o dataset diretamente a partir de uma URL pública. O dataset é armazenado em um DataFrame do pandas (`df`) para posterior análise.


In [ ]:
# Bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações gráficas
sns.set(style="whitegrid")
%matplotlib inline

# Carregando o dataset via URL
url = 'https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv'
df = pd.read_csv(url)

# Cópia de segurança do dataset original (para comparações futuras)
df_original = df.copy()


## 2. Entendimento dos Dados

Nesta etapa, realizamos uma exploração inicial da estrutura do dataset para obter uma visão geral da sua composição e qualidade.

O objetivo principal é compreender:

- O tamanho da base de dados
- A presença de valores ausentes (NaN ou NaT)
- Os tipos de dados de cada variável
- As primeiras e últimas linhas da base
- Estatísticas descritivas básicas

Essas análises são essenciais para orientar as decisões nas etapas seguintes de preparação e modelagem.


### 🔎 Verificando o tamanho do dataset

O método `df.shape` retorna uma tupla com o número de linhas e colunas do DataFrame. Essa é uma informação fundamental para entendermos a dimensão da base de dados que será analisada e modelada.

- O primeiro valor representa o número de registros (transações).
- O segundo valor representa o número de variáveis (colunas).

Essa análise nos dá uma ideia inicial da escala do problema.


In [ ]:
# Verificando o número de linhas e colunas
df.shape


### 🧾 Visualizando as primeiras linhas do dataset

O método `df.head()` exibe, por padrão, as **cinco primeiras linhas** do DataFrame. Isso permite observar:

- A estrutura dos dados
- A ordem das colunas
- Exemplos reais de valores presentes
- Possíveis inconsistências ou padrões

É uma das primeiras formas de "enxergar" o conteúdo da base.


In [ ]:
# Visualizando as primeiras 5 linhas do dataset
df.head()


### 📄 Visualizando as últimas linhas do dataset

O método `df.tail()` retorna, por padrão, as **cinco últimas linhas** do DataFrame.  
Isso é útil para verificar se há algum comportamento atípico no final da base de dados, como:

- Registros incompletos
- Campos zerados ou nulos
- Mudanças de padrão

Também complementa a visualização iniciada com `df.head()`.


In [ ]:
# Visualizando as 5 últimas linhas do dataset
df.tail()


### 🧱 Verificando estrutura e tipos de dados com `.info()`

O método `df.info()` fornece um resumo das colunas do DataFrame, incluindo:

- Número total de entradas (linhas)
- Nome de cada coluna
- Quantidade de valores não nulos
- Tipo de dado de cada coluna

É fundamental para:

- Identificar valores ausentes (NaN ou NaT)
- Verificar a consistência dos tipos de dados (ex: `float64`, `int64`)
- Estimar o uso de memória do dataset


In [ ]:
# Resumo da estrutura do DataFrame
df.info()


### 📊 Estatísticas descritivas com `.describe()`

O método `df.describe()` gera estatísticas descritivas para as colunas numéricas do DataFrame, como:

- Média
- Desvio padrão
- Valores mínimos e máximos
- Quartis (Q1, Q2 - mediana, Q3)

Esses dados ajudam a identificar:

- Distribuições assimétricas
- Outliers (valores extremos)
- Escalas distintas entre variáveis

É especialmente útil antes de aplicar transformações como normalização ou padronização.


In [ ]:
# Estatísticas descritivas das variáveis numéricas
df.describe()


## 3. Pré-processamento de Texto (Análise Auxiliar)

Como solicitado pelo professor, incluímos nesta etapa a aplicação de técnicas de pré-processamento textual.  
Embora o dataset principal (`creditcard.csv`) não contenha campos de texto, construímos um **texto explicativo do projeto** como corpus base.

As etapas aplicadas foram:

- Minificação do texto (lowercase)
- Remoção de pontuação, acentuação e dígitos
- Remoção de stopwords
- Stemming com `PorterStemmer`
- Vetorização com TF-IDF

A seguir, mostramos cada etapa aplicada ao texto.


### 📚 Importação das bibliotecas


In [ ]:
import re
import string
import numpy as np
import pandas as pd

from unidecode import unidecode

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer


### 🧾 Texto explicativo do projeto (usado como corpus)


In [ ]:
texto_projeto = """Este projeto tem como objetivo desenvolver um sistema inteligente capaz de detectar fraudes em transações financeiras 
por meio de técnicas avançadas de aprendizado de máquina supervisionado. A solução proposta busca identificar padrões 
anômalos em grandes volumes de dados, utilizando etapas rigorosas de pré-processamento textual, vetorização e 
modelagem preditiva. O desempenho dos modelos será avaliado com base em métricas como acurácia, precisão, recall e 
F1-score, assegurando sua robustez e aplicabilidade em contextos reais. Destinado a instituições financeiras, o 
sistema funcionará como uma ferramenta preventiva, reforçando a segurança digital e contribuindo para a mitigação de 
perdas decorrentes de atividades fraudulentas.
"""


### 🧼 Pré-processamento inicial (minúsculas, pontuação, acentos, dígitos)


In [ ]:
# 1. Minúsculas
texto = texto_projeto.lower()

# 2. Remoção de pontuação
punctuation = string.punctuation
trantab = str.maketrans(punctuation, len(punctuation) * ' ')
texto = texto.translate(trantab)

# 3. Remoção de acentos
texto = unidecode(texto)

# 4. Remoção de dígitos
texto = re.sub(r'\d+', '', texto)


### ❌ Remoção de stopwords (com `try-except`)


In [ ]:
try:
    stopwords_list = stopwords.words('portuguese')
except:
    nltk.download('stopwords')
    stopwords_list = stopwords.words('portuguese')

palavras = texto.split()
palavras_filtradas = [palavra for palavra in palavras if palavra not in stopwords_list and len(palavra) > 1]
texto_limpo = " ".join(palavras_filtradas)

print(texto_limpo)


### 🧪 Stemming com `PorterStemmer`


In [ ]:
stemmer = PorterStemmer()
texto_stemmed = " ".join([stemmer.stem(palavra) for palavra in texto_limpo.split()])

print(texto_stemmed)


### 🧠 Vetorização com TF-IDF


In [ ]:
corpus = np.array([
    texto_stemmed,
    "sistema inteligente detectar padrao comportamento transacao financeira"
])

vectorizer = TfidfVectorizer()
tfidf_result = vectorizer.fit_transform(corpus)

df_tfidf = pd.DataFrame.sparse.from_spmatrix(tfidf_result, columns=vectorizer.get_feature_names_out())
df_tfidf


## 4. Análise Exploratória de Dados (EDA)

Nesta etapa, buscamos explorar visualmente os dados para compreender a distribuição de valores, detectar padrões, identificar valores discrepantes (outliers) e verificar o comportamento da variável-alvo `Class`.

Além disso, serão aplicadas transformações simples para comparar o "antes e depois" dos dados, conforme exigido na atividade da disciplina.


### 🎯 Distribuição da variável-alvo `Class`

Vamos visualizar a proporção entre transações legítimas (`Class = 0`) e fraudulentas (`Class = 1`).


In [ ]:
sns.countplot(x='Class', data=df, palette='Set2')
plt.title('Distribuição das Transações (0 = Legítima, 1 = Fraude)')
plt.xlabel('Classe')
plt.ylabel('Contagem')
plt.show()


A maior parte das transações é legítima (`Class = 0`). Apenas **0,172%** dos registros representam fraudes (`Class = 1`), o que confirma que o dataset é altamente desbalanceado.  
Isso influenciará diretamente na escolha dos algoritmos e nas métricas de avaliação utilizadas posteriormente.


### 💶 Análise da variável `Amount`

Vamos analisar a distribuição dos valores das transações antes de qualquer transformação, utilizando:

- Histograma (para ver concentração)
- Boxplot (para observar outliers)


In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(df['Amount'], bins=50, kde=True)
plt.title('Distribuição dos Valores das Transações (Antes da Normalização)')
plt.xlabel('Valor (€)')
plt.ylabel('Frequência')
plt.show()


In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['Amount'])
plt.title('Boxplot do Valor das Transações (Antes da Normalização)')
plt.xlabel('Valor (€)')
plt.show()


Os gráficos mostram que a maioria das transações possui valores baixos, concentrados abaixo de €100.  
Entretanto, existem outliers (valores muito altos), o que pode distorcer análises e impactar algoritmos sensíveis à escala, como regressão logística e KNN.

Por isso, será necessário aplicar uma transformação de escala nos valores.


### 🔧 Normalização do valor da transação (`Amount`)

Alguns algoritmos de Machine Learning são sensíveis à escala dos dados. Como a variável `Amount` apresenta uma grande variação e outliers, vamos aplicar a **normalização com `StandardScaler`**, que transforma os dados para que tenham média 0 e desvio padrão 1.

Dessa forma, o algoritmo será menos influenciado por diferenças de escala.


In [ ]:
from sklearn.preprocessing import StandardScaler

# Instanciando o normalizador
scaler = StandardScaler()

# Aplicando ao Amount
df['Amount_Scaled'] = scaler.fit_transform(df[['Amount']])

# Verificando estatísticas antes e depois
df[['Amount', 'Amount_Scaled']].describe()


### 📊 Comparação “Antes e Depois” com Gráficos

Vamos comparar a distribuição da variável `Amount` antes e depois da transformação.


In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

# Boxplot original
sns.boxplot(x=df['Amount'], ax=axs[0], color='skyblue')
axs[0].set_title('Antes da Normalização')
axs[0].set_xlabel('Amount (€)')

# Boxplot escalado
sns.boxplot(x=df['Amount_Scaled'], ax=axs[1], color='lightgreen')
axs[1].set_title('Depois da Normalização')
axs[1].set_xlabel('Amount Scaled')

plt.suptitle('Boxplots Comparativos - Amount Original vs Normalizado')
plt.tight_layout()
plt.show()


### 📊 Histograma do valor normalizado das transações

Para complementar a visualização, vamos analisar o histograma da variável `Amount_Scaled` após a transformação com `StandardScaler`.


In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(df['Amount_Scaled'], bins=50, kde=True, color='green')
plt.title('Distribuição do Valor das Transações Após Normalização')
plt.xlabel('Amount Scaled')
plt.ylabel('Frequência')
plt.show()


A transformação com `StandardScaler` centralizou os dados de `Amount_Scaled` em torno de zero e reduziu a variância.  
Isso facilita a aprendizagem dos algoritmos e mitiga a influência de outliers extremos.  
O valor original foi mantido em `Amount` para fins de referência, enquanto o modelo utilizará `Amount_Scaled`.
